1. Problema: 
    - Encontrar os melhores clientes

3. *Descrição das Colunas do Dataset*

| Coluna       | Descrição |
|-------------|-----------|
| `invoiceno`  | Número da fatura. Um identificador único para cada transação realizada. |
| `stockcode`  | Código do produto. Um identificador único para cada item vendido. |
| `description` | Descrição do produto. O nome ou detalhes do item comprado. |
| `quantity`   | Quantidade de itens comprados em uma única transação. |
| `invoicedate` | Data e hora da compra, indicando quando a transação ocorreu. |
| `unitprice`  | Preço unitário do produto, ou seja, o valor de um único item. |
| `customerid` | Identificação do cliente. Um número único que representa cada comprador. |
| `country`    | País de origem do cliente que realizou a compra. |


1. Quem são as pessoas elegíveis para participar do programa de Insiders?
    - O que é ser elegível? O que são clientes de maior "valor"?
    - Faturamento:
      - Alto Ticket Médio.*
      - Alto LTV.*
      - Baixa Recência.*
      - Alto basket size.*
      - Baixa Probabilidade de Churn.*
      - Alta Previsão LTV.*
      - Alta Propensão de Compra.*
    
    - Custo:
      - Baixa taxa de devolução.
     
     - Experiência de Compra:
       - Média Alta das Avaliação

2. Quantos clientes farão parte do grupo?
   - Número Total de Clientes
   - % do grupo Insiders
  
3. Quais as principais caracteristicas desses clientes?
    - Escrever Características do Cliente:
      - Idade
      - Localização
    - Escrever Características do Consumo.
      - Atributos da Clusterização.
  
4. Qual a porcentagem de contribuição do faturameto vinda do Insiders?
   - Faturamento total do ano
   - Faturamento do grupo Insiders
  
5. Qual a expectativa de faturamento desse grupo para os próximos meses?
    - LTV do grupo Insiders
    - Análise de Cohort*
  
6. Quais as condições para uma pessoa ser elegível ao Insiders?
   - Definir a periodicidade ( 1 mes, 3 meses )
   - A pessoa precisa ser similar com uma pessoa do grupo.
  
7. Quais as condições para uma pessoa ser removida do Insiders?
    - Definir a periodicidade ( 1 mes, 3 meses )
   - A pessoa precisa ser não similar com uma pessoa do grupo.
8. Qual a garantia que o programa Insiders é melhor que o restante da base?
   - Teste A/B
   - Teste A/B Bayesiano
   - Teste de Hipoteses
9.  Quais ações o time de marketing pode realizar para aumentar o faturamento?  
    - Desconto
    - Preferencia de Compra

# 1.0 IMPORTS

In [3]:
import pandas   as pd
import numpy    as np

import matplotlib.pyplot    as plt
import seaborn              as sns
from plotly import express as px
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.model_selection import train_test_split
from sklearn import cluster as c
from sklearn import metrics as m
#import lux

import umap.umap_ as umap

import re

import warnings
warnings.filterwarnings('ignore')

ImportError: cannot import name 'templates' from partially initialized module 'numba.core.typing' (most likely due to a circular import) (C:\Users\s1294807\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numba\core\typing\__init__.py)

## 1.1 Loading Data

In [ ]:
df = pd.read_csv('./data/ecommerce.csv', encoding="ISO-8859-1")
df = df.drop('Unnamed: 8', axis=1)

## 1.2 Helper Functions

In [ ]:
def descriptive_analysis(num_attrs):

    # central tendency - mean, media
    ct1 = pd.DataFrame(num_attrs.apply(np.mean)).T
    ct2 = pd.DataFrame(num_attrs.apply(np.median)).T

    # disperation - std, min, max, range, skew, kurtosis
    d1 = pd.DataFrame(num_attrs.apply(np.std)).T
    d2 = pd.DataFrame(num_attrs.apply(min)).T
    d3 = pd.DataFrame(num_attrs.apply(max)).T
    d4 = pd.DataFrame(num_attrs.apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(num_attrs.apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(num_attrs.apply(lambda x: x.kurtosis())).T

    # concatenate
    m = pd.concat([d2,d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
    m.columns = ['attrs', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']

    return m

# 2.0 DATA DESCRIPTION

In [ ]:
df2 = df.copy()

## 2.1 Rename columns

In [ ]:
df2.columns = [x.lower() for x in df2.columns]
columns = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date', 'unit_price', 'customer_id', 'country']
df2.columns = columns
df2.sample()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
525412,580619,23077,DOUGHNUT LIP GLOSS,40,3-Dec-17,1.25,15053.0,United Kingdom


## 2.2 Data Dimension

In [ ]:
print('Columns:', df2.shape[1])
print('Rows:', df2.shape[0])

Columns: 8
Rows: 541909


## 2.3 Data Types

In [ ]:
df2.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 2.4 Check NA

In [ ]:
df2.isnull().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

In [ ]:
df2[df2['customer_id'].isna()]

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
622,536414,22139,NaN,56,29-Nov-16,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,29-Nov-16,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,29-Nov-16,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,29-Nov-16,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,29-Nov-16,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,7-Dec-17,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,7-Dec-17,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,7-Dec-17,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,7-Dec-17,10.79,NaN,United Kingdom


Unfortunately, for this problem, customerid is very important. And since it is not possible to identify these columns, I will remove them. Next cycle I will tratment this problem.

## 2.5 Replace NA

In [ ]:
df2_missing_na = df2.loc[df2['customer_id'].isna(), :]
df2_not_missing_na = df2.loc[~df2['customer_id'].isna(), :]

In [ ]:
# create reference
df_backup = pd.DataFrame(df2_missing_na['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup), 1)

# merge original with reference dataframe
df2 = pd.merge(df2, df_backup, on='invoice_no', how='left')
df2.head()

# coalesce
df2['customer_id'] = df2['customer_id_x'].combine_first(df2['customer_id_y'])

# drop extra columns
df2 = df2.drop(columns=['customer_id_x', 'customer_id_y'], axis=1)

df2.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,United Kingdom,17850.0
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,United Kingdom,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,United Kingdom,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,United Kingdom,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,United Kingdom,17850.0


In [ ]:
df2.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

## 2.6 Change dtypes

In [ ]:
df2.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
country          object
customer_id     float64
dtype: object

In [ ]:
# invoice data
df2['invoice_date'] = pd.to_datetime(df2['invoice_date'], format='%d-%b-%y')

# customer id
df2['customer_id'] = df2['customer_id'].astype(int)

## 2.7 Descriptive Statistical

In [ ]:
num_attributes = df2.select_dtypes(include=['int64', 'float64', 'int32', 'float32'])
categorical_attributes = df2.select_dtypes(exclude=['int64', 'float64', 'int32', 'float32', 'datetime64[ns]'])

### 2.7.1 Numerical Attributs

In [ ]:
descriptive_analysis(num_attributes)

,attrs,min,max,range,mean,median,std,skew,kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.080957,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611114,2.08,96.759764,186.506972,59005.719097
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.408666,0.487449,-0.804287


#### <font color='red'> 2.7.1.1 Numerical Attributs - Investigating </font>

1. Quantity negativa ( pode ser devolução )
2. Preço Unitário igual a zero ( pode ser promoção ? )

### 2.7.2 Categorical Attributs

**Invoice No**

In [ ]:
# problema: temos invoice com letras e numeros
# df2['invoice_no'].astype(int)

# identificacao
df_letter_invoices = df2.loc[df2['invoice_no'].apply(lambda x: bool(re.search('[^0-9]+', x))), : ]

print(f'Total number of invoice: {len(df_letter_invoices)}')
print('Total number of negative quantify: {}'.format(len(df_letter_invoices[df_letter_invoices['quantity'] < 0])))


Total number of invoice: 9291
Total number of negative quantify: 9288


**Stock Code**

In [ ]:
# check stock code only characters
df2.loc[df2.stock_code.apply(lambda x: bool( re.search('^[a-zA-Z]+$', x))), 'stock_code'].unique()

#acao:
## 1. remove stock_code in ['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK']

array(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

**Description**

In [ ]:
df2.head()

#acao: dele description

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2016-11-29,3.39,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2016-11-29,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2016-11-29,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2016-11-29,3.39,United Kingdom,17850


**Country**

In [ ]:
df2.country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [ ]:
df2['country'].value_counts(normalize=True)

country
United Kingdom          0.914320
Germany                 0.017521
France                  0.015790
EIRE                    0.015124
Spain                   0.004674
Netherlands             0.004375
Belgium                 0.003818
Switzerland             0.003694
Portugal                0.002803
Australia               0.002323
Norway                  0.002004
Italy                   0.001482
Channel Islands         0.001399
Finland                 0.001283
Cyprus                  0.001148
Sweden                  0.000853
Unspecified             0.000823
Austria                 0.000740
Denmark                 0.000718
Japan                   0.000661
Poland                  0.000629
Israel                  0.000548
USA                     0.000537
Hong Kong               0.000531
Singapore               0.000423
Iceland                 0.000336
Canada                  0.000279
Greece                  0.000269
Malta                   0.000234
United Arab Emirates    0.000125
Eu

In [ ]:
df2[['customer_id', 'country']].drop_duplicates().groupby('country').count().reset_index().sort_values('customer_id', ascending=False)

,country,customer_id
36,United Kingdom,7587
14,Germany,95
13,France,90
10,EIRE,44
31,Spain,31
3,Belgium,25
33,Switzerland,24
27,Portugal,20
19,Italy,15
16,Hong Kong,15


In [ ]:
categorical_attributes.head()

,invoice_no,stock_code,description,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
1,536365,71053,WHITE METAL LANTERN,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,United Kingdom


In [ ]:
# invoice number

#categorical_attributes.invoice_no.astype(int)
len( categorical_attributes.loc[categorical_attributes.invoice_no.apply(lambda x: bool(re.search('[^0-9]+', x ))), 'invoice_no'].drop_duplicates() )

3839

In [ ]:
# # stock code
# at_least_one_string = len(categorical_attributes.loc[categorical_attributes['stock_code'].apply(lambda x: bool(re.search('[^0-9]+', x))), 'stock_code'])
# print(at_least_one_string)

#
categorical_attributes.loc[categorical_attributes['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))), 'stock_code'].unique()

#df2.loc[categorical_attributes['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]$', x))), :].head(30)

array(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

In [ ]:
df2['country'].value_counts(normalize=True).head()

country
United Kingdom    0.914320
Germany           0.017521
France            0.015790
EIRE              0.015124
Spain             0.004674
Name: proportion, dtype: float64

In [ ]:
df2[['customer_id', 'country']].drop_duplicates().groupby('country').count().reset_index().sort_values('customer_id', ascending=False).head()

,country,customer_id
36,United Kingdom,7587
14,Germany,95
13,France,90
10,EIRE,44
31,Spain,31


# 3.0 FEATURE FILTERING

In [ ]:
df3 = df2.copy()

In [ ]:
# unit price > 0.0
df3 = df3.loc[df3['unit_price'] > 0.04, :]

# categorical attributes
df3 = df3[~df3['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY','DCGSSGIRL', 'PADS', 'B', 'CRUK'])]

# description
df3 = df3.drop(columns='description', axis=1)

# map - 
df3 = df3[~df3['country'].isin(['European Community', 'Unspecified'])]

# quantity
df3_returns = df3.loc[df3['quantity'] < 0, :]
df3_purchase = df3.loc[df3['quantity'] > 0, :]

# 4.0 FEATURE ENGINEERING

In [ ]:
# Feature Ideas:
## Moving Average - 7d, 14d, 21d
## Quantidade de compras por mes, antes do dia 15 e depous do dia 15
## average financial
## basket size * price - quantidade de preços por cesta

In [ ]:
df4 = df3.copy()

## 4.1 Feature Creation

In [ ]:
df_ref = df4.drop(['invoice_no', 'stock_code', 'quantity', 
                   'invoice_date', 'unit_price', 'country'], axis=1).drop_duplicates(ignore_index=True)

### 4.1.1 Gross Revenue

In [ ]:
# faturamento: quantity * price
df3_purchase['gross_revenue'] = df3_purchase['quantity'] * df3_purchase['unit_price']

# monetary
df_monetary = df3_purchase[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, how='left', on='customer_id')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
dtype: int64

### 4.1.2 Recency - Day from last purchase

In [ ]:
# recency
df_recency = df3_purchase[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df3_purchase['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge(df_ref, df_recency, how='left', on='customer_id')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
dtype: int64

### 4.1.3 Quantity of purchased 

In [ ]:
# numero de produtos
df_frequency = df3_purchase[['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index()
df_ref = pd.merge(df_ref, df_frequency, how='left', on='customer_id')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
invoice_no       91
dtype: int64

### 4.1.4 Quantity of products purchased

In [ ]:
# numero de produtos
df_frequency = df3_purchase[['customer_id', 'quantity']].drop_duplicates().groupby('customer_id').count().reset_index()
df_ref = pd.merge(df_ref, df_frequency, how='left', on='customer_id')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
invoice_no       91
quantity         91
dtype: int64

### 4.1.5 Average Ticket Value

In [ ]:
# avg ticket
df_avg_ticket = df3_purchase.loc[:, ['customer_id', 'gross_revenue']].groupby('customer_id').mean().reset_index().rename(columns={'gross_revenue':'avg_ticket'})
df_ref = pd.merge(df_ref, df_avg_ticket, how='left', on='customer_id')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
invoice_no       91
quantity         91
avg_ticket       91
dtype: int64

### 4.1.6 Average Recency Days

In [ ]:
# Average recency days
df_aux = df3[['customer_id', 'invoice_date']].drop_duplicates().sort_values(['customer_id', 'invoice_date'])
df_aux['next_customer_id'] = df_aux['customer_id'].shift() # next customer
df_aux['previous_date'] = df_aux['invoice_date'].shift() # next invoice date

df_aux['avg_recency_days'] = df_aux.apply(lambda x: (x['invoice_date'] - x['previous_date']).days if x['customer_id'] == x['next_customer_id'] else np.nan, axis=1)

df_aux = df_aux.drop( ['invoice_date', 'next_customer_id', 'previous_date'], axis=1).dropna()

# average recency
df_avg_recency_days = df_aux.groupby('customer_id').mean().reset_index()

# merge
df_ref = pd.merge(df_ref, df_avg_recency_days, on='customer_id', how='left')
df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
invoice_no            91
quantity              91
avg_ticket            91
avg_recency_days    2816
dtype: int64

### 4.1.7 Frequency Purchase

In [ ]:
df_aux = ( df3[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                  .groupby('customer_id')
                                                  .agg(max__ = ('invoice_date', 'max'),
                                                       min__ = ('invoice_date', 'min'),
                                                       days__ = ('invoice_date', lambda x: ( x.max() - x.min() ).days ),
                                                       buy__ = ('invoice_no', 'count')) ).reset_index()
# frequency
df_aux['frequency'] = df_aux[['buy__', 'days__']].apply(lambda x: x['buy__'] / x['days__'] if x['days__'] != 0 else 0, axis=1)

# merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
invoice_no            91
quantity              91
avg_ticket            91
avg_recency_days    2816
frequency              0
dtype: int64

### 4.1.8 Number of Returns

In [ ]:
df3_returns

,invoice_no,stock_code,quantity,invoice_date,unit_price,country,customer_id
154,C536383,35004C,-1,2016-11-29,4.65,United Kingdom,15311
235,C536391,22556,-12,2016-11-29,1.65,United Kingdom,17548
236,C536391,21984,-24,2016-11-29,0.29,United Kingdom,17548
237,C536391,21983,-24,2016-11-29,0.29,United Kingdom,17548
238,C536391,21980,-24,2016-11-29,0.29,United Kingdom,17548
...,...,...,...,...,...,...,...
540448,C581490,22178,-12,2017-12-07,1.95,United Kingdom,14397
540449,C581490,23144,-11,2017-12-07,0.83,United Kingdom,14397
541715,C581568,21258,-5,2017-12-07,10.95,United Kingdom,15311
541716,C581569,84978,-1,2017-12-07,1.25,United Kingdom,17315


In [ ]:
df_returns = df3_returns[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity':'qtde_returns'})
df_returns['qtde_returns'] = df_returns['qtde_returns'] * -1

df_ref = pd.merge(df_ref, df_returns, how='left', on='customer_id')
df_ref.loc[df_ref['qtde_returns'].isna(), 'qtde_returns'] = 0

df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
invoice_no            91
quantity              91
avg_ticket            91
avg_recency_days    2816
frequency              0
qtde_returns           0
dtype: int64

### 4.1.9 Basket Size - quantidade de itens por cesta ( quantity )

In [ ]:
df_aux = (df3_purchase.loc[:, ['customer_id', 'invoice_no', 'quantity']].groupby('customer_id')
                                                                 .agg( 
                                                                    n_purchase=('invoice_no', 'nunique'),
                                                                    n_products=('quantity', 'sum'))
                                                                 .reset_index()
                                                                 )

df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

df_ref = pd.merge(df_ref, df_aux[['customer_id', 'avg_basket_size']], how='left', on='customer_id')
df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
invoice_no            91
quantity              91
avg_ticket            91
avg_recency_days    2816
frequency              0
qtde_returns           0
avg_basket_size       91
dtype: int64

### 4.2.0 unique basket size - quantidade de produtos distintos por compra

In [ ]:
df_aux = (df3_purchase.loc[:, ['customer_id', 'invoice_no', 'stock_code']].groupby('customer_id')
                                                                 .agg( 
                                                                    n_purchase=('invoice_no', 'nunique'),
                                                                    n_products=('stock_code', 'count'))
                                                                 .reset_index()
                                                                 )

df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

df_ref = pd.merge(df_ref, df_aux[['customer_id', 'avg_unique_basket_size']], how='left', on='customer_id')
df_ref.isna().sum()

customer_id                  0
gross_revenue               91
recency_days                91
invoice_no                  91
quantity                    91
avg_ticket                  91
avg_recency_days          2816
frequency                    0
qtde_returns                 0
avg_basket_size             91
avg_unique_basket_size      91
dtype: int64

# 5.0 EDA ( Exploratory Data Analysis )

In [ ]:
df4 = df_ref.dropna().copy()
df4.isna().sum()

customer_id               0
gross_revenue             0
recency_days              0
invoice_no                0
quantity                  0
avg_ticket                0
avg_recency_days          0
frequency                 0
qtde_returns              0
avg_basket_size           0
avg_unique_basket_size    0
dtype: int64

## 4.1 Univariate Analysis

In [2]:
df4

NameError: name 'df4' is not defined

## 4.2 Bivariate Analysis

# 6.0 Data Preparation

In [ ]:
from sklearn import preprocessing as pp

In [ ]:
df5 = df4.copy()

In [ ]:
#ss = pp.StandardScaler()
ss = pp.MinMaxScaler()


df5['gross_revenue'] = ss.fit_transform( df5[['gross_revenue']] )
df5['recency_days'] = ss.fit_transform( df5[['recency_days']] )
df5['invoice_no'] = ss.fit_transform( df5[['invoice_no']] )
df5['avg_ticket'] = ss.fit_transform( df5[['avg_ticket']] )
df5['returns'] = ss.fit_transform( df5[['returns']] )


KeyError: "None of [Index(['returns'], dtype='object')] are in the [columns]"

In [ ]:
sns.displot(np.log ( df5['gross_revenue']))

# 7.0 Feature Selection

In [ ]:
df6 = df5.copy()

# 8.0 Hyperparameter Fine-Tunning

In [ ]:
df7 = df6.drop(columns=['customer_id'])
X = df7.copy()

In [ ]:
clusters = [2,3,4,5,6,7]

## 7.1 Within-Cluster Sum of Square ( WSS )

In [ ]:
wss = []
for k in clusters:
    # model definition
    kmeans = c.KMeans(init='random', n_clusters=k, n_init=10, max_iter=300, random_state=42)
    # model training
    kmeans.fit(X)
    # validation
    wss.append(kmeans.inertia_)
# plot wss
plt.plot(clusters, wss, linestyle='--', marker='o', color='b')
plt.xlabel('K');
plt.ylabel('Within-Cluster Sum of Square');
plt.title('WSS vs K');

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
kmeans = KElbowVisualizer(c.KMeans(), k=clusters, timings=False)
kmeans.fit(X)
kmeans.show();

## 7.2 Silhouette Score

In [ ]:
kmeans = KElbowVisualizer(c.KMeans(), k=clusters, metric='silhouette', timings=False)
kmeans.fit(X)
kmeans.show();

## 7.2 Silhouette Analysis

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(25,18))
for k in clusters:
    km = c.KMeans(n_clusters=k, init='random', n_init=10, max_iter=100,  random_state=42)
    q, mod = divmod(k, 2)
    visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(X)
    visualizer.finalize()

# 9.0 Model Training

In [ ]:
df8 = df7.copy()

## 8.1 K-Means

In [ ]:
# model definition
k = 3
kmeans = c.KMeans(init='random', n_clusters=k, n_init=10, max_iter=300, random_state=42)

# model training 
kmeans.fit(X)

# clustering
labels = kmeans.labels_

## 8.2 Cluster Validation

In [ ]:
print('WSS Value: {}'.format(kmeans.inertia_))
print('SS Value: {:.2f}'.format(m.silhouette_score(X, labels, metric='euclidean')))


# 10.0 Cluster Analysis

In [ ]:
df6['cluster'] = labels
df9 = df6.copy()
df9.head()

## 9.1 Visualization Inspection

In [ ]:
visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick')
visualizer.fit(X)
visualizer.finalize()

## 9.2 2dplot

In [ ]:
df9.head()

In [ ]:
df_viz = df9.drop(columns='customer_id', axis=1)
sns.pairplot(df_viz, hue='cluster')

## 9.3 UMAP - t-SNE

In [ ]:
reducer = umap.UMAP(n_neighbors=90, random_state=42)
embedding = reducer.fit_transform(X)

df_viz['embedding_x'] = embedding[:, 0]
df_viz['embedding_y'] = embedding[:, 1]

sns.scatterplot(x='embedding_x', y='embedding_y', hue='cluster', palette=sns.color_palette('hls', n_colors=len( df_viz['cluster'].unique())), data=df_viz)

## Plot 3D

In [ ]:
df9 = df9.rename(columns={'invoice_no':'frequency'})

fig = px.scatter_3d(df9, x='recency_days', y='frequency', z='gross_revenue', color='cluster')
fig.show();

## 9.2 Cluster Profile

In [ ]:
df9.head()

In [ ]:
# number of cluster
df_cluster = df9[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customer'] = 100*(df_cluster['customer_id'] / df_cluster['customer_id'].sum())

# avg gross revenue
df_avg_gross_revenue = df9[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, how='inner', on='cluster')

# avg recency days
df_avg_recency_days = df9[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency_days, how='inner', on='cluster')

# avg invoice_no
df_avg_invoice_no = df9[['frequency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_invoice_no, how='inner', on='cluster')

# avg ticket
df_avg_ticket = df9[['avg_ticket', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_ticket, how='inner', on='cluster')

df_cluster

# 11 Model Translation